## Visualizing and Understanding your Data in R

- **Kernel**: `conda_mxnet_p36` when running `import mxnet` then `R` for everything else

### Import MXNet

In [1]:
# ensure kernel is conda_mxnet_p36

In [2]:
import mxnet

In [ ]:
# Change kernel to R

In [1]:
print('hello from R')

[1] "hello from R"


In [2]:
library('reticulate')

In [3]:
import.MXNet.attempt = function() {
    mxnet <- NULL

    tryCatch({
        mxnet <- import('mxnet')
        print('Successfully loaded MXNet')
    }, error = function(e) {
        print('Error loading MXNet')
    })  
    
    return(mxnet)
}

import.MXNet = function() {
    mxnet = NULL
    
    for (i in 1:10) {
        mxnet <- import.MXNet.attempt()
        if (!is.null(mxnet)) {
            return(mxnet)
        }
    }
    
    return(mxnet)
}

In [4]:
mxnet <- import.MXNet()

[1] "Error loading MXNet"
[1] "Successfully loaded MXNet"


In [5]:
mxnet

Module(mxnet)

In [32]:
mxnet['__version__']

[1] "1.5.1"

### Load Model with Apache MXNet

In [7]:
json <- import("json") 
builtins <- import_builtins()

In [8]:
model_data <- readRDS(file="tmp/estimator$model_data")

In [9]:
model_data

[1] "s3://sagemaker-cookbook-bucket/chapter01/output/linear-learner-2021-03-13-09-45-05-295/output/model.tar.gz"

In [10]:
cmd <- function(bash_command) {
    output <- system(bash_command, intern=TRUE)
    last_line = ""
    
    for (line in output) { 
        cat(line)
        cat("\n")
        last_line = line 
    }
    
    return(last_line) 
}

In [11]:
cmd(paste("aws s3 cp", model_data, "tmp/model.tar.gz"))

download: s3://sagemaker-cookbook-bucket/chapter01/output/linear-learner-2021-03-13-09-45-05-295/output/model.tar.gz to tmp/model.tar.gz


[1] "Completed 851 Bytes/851 Bytes (7.9 KiB/s) with 1 file(s) remaining\rdownload: s3://sagemaker-cookbook-bucket/chapter01/output/linear-learner-2021-03-13-09-45-05-295/output/model.tar.gz to tmp/model.tar.gz"

In [12]:
cmd("tar -xzvf tmp/model.tar.gz")

model_algo-1


[1] "model_algo-1"

In [13]:
cmd("unzip model_algo-1")

Archive:  model_algo-1
 extracting: additional-params.json  
 extracting: mx-mod-0000.params      
 extracting: manifest.json           
 extracting: mx-mod-symbol.json      


[1] " extracting: mx-mod-symbol.json      "

In [15]:
sym_json <- json$load(builtins$open('mx-mod-symbol.json')) 
sym_json_string = json$dumps(sym_json)

outputs <- mxnet$sym$load_json(sym_json_string) 
inputs <- mxnet$sym$var('data')

model <- mxnet$gluon$nn$SymbolBlock(outputs=outputs,
                                    inputs=inputs) 

model$load_parameters('mx-mod-0000.params', allow_missing=TRUE)
model$initialize()

In [16]:
mxnet_predict <- function(x, target_model=model) {
    output = target_model(mxnet$nd$array(as.list(x)))[0]$asscalar()
    
    return(output) 
}

In [17]:
mxnet_predict("42")

[1] 1209.774

In [19]:
extract_weight_and_bias <- function(model) {
    params <- model$collect_params()
    weight <- params['fc0_weight']$data()[0]$asscalar() 
    bias <- params['fc0_bias']$data()[0]$asscalar()
    
    return (list(weight=weight, bias=bias)) 
}

In [20]:
weight_and_bias <- extract_weight_and_bias(model)
weight_and_bias 

$weight
[1] 7.446002

$bias
[1] 897.0424

In [21]:
saveRDS(weight_and_bias, file="tmp/weight_and_bias")

In [23]:
to.be.deleted <- c(
  'additional-params.json',
  'manifest.json',
  'model_algo-1',
  'mx-mod-symbol.json',
  'mx-mod-0000.params'
)

In [25]:
for (file in to.be.deleted) {
    delete_command <- paste("rm -f", file)
    cmd(delete_command)
}